In [1]:
import pandas as pd
df = pd.read_csv('/Users/margarethe/Downloads/Data Safety_May 23, 2022_15.15.csv')

In [2]:
answers = pd.read_csv('/Users/margarethe/Downloads/Matcha definition-example table - Data collection.csv')

In [3]:
a_dict = dict()
questions = answers.loc[:,"Example Scenario in Survey"]
correct = answers.loc[:,"Correct Answer"]
for i in range(0, len(questions)):
    a_dict[questions[i]] = correct[i]

In [4]:
analysis = dict()
valcounts = list(df.columns.values.tolist())
for i in range(7, len(valcounts)):
    correct = 0
    incorrect = 0
    answers = df[list(df.columns.values.tolist())[i]].value_counts().keys().tolist()
    counts = df[list(df.columns.values.tolist())[i]].value_counts().tolist()
    if a_dict.get(valcounts[i], 42) == 42: continue
    for j in range(len(answers)):
        if answers[j] == a_dict[valcounts[i]]:
            correct = counts[j]
        else:
            incorrect += counts[j]
    analysis[valcounts[i]] = (correct, incorrect)

In [5]:
print("Section 1: \n")
for val in analysis:
    if "user/device physical location to area greater" in val:
        print("Section 2: \n")
    (correct, incorrect) = analysis[val]
    print(val + ": ")
    print("Correct= " + str(correct) + ", Incorrect= " + str(incorrect))
    print("Rate= " + str((correct/(correct+incorrect))))
    print("\n")

Section 1: 



In [40]:
df = pd.read_csv('/Users/margarethe/Downloads/Data Safety_May 23, 2022_15.15.csv')
descriptionsDict = dict()
start = df.columns.get_loc("Q135")
end = df.columns.get_loc("Q101")
for i in range(start, end + 1, 1):
    key = df.columns[i]
    entry = df.iat[0, i]
    descriptionsDict[key] = entry
typeDict = dict()
datatypes = {'Q162', 'Q135', 'Q109', 'Q108', 'Q128', 'Q16',	'Q17', 'Q129', 'Q19', 'Q26', 'Q20', 'Q122', 'Q22', 'Q21', 'Q23', 'Q132', 'Q133', 'Q24', 'Q123', 'Q125', 'Q25', 'Q127', 'Q27', 'Q130'}
section1 = ['Data Collection', 'Data Sharing', 'Both', 'Neither']
section2 = ['Location', 'Personal Information', 'Financial Information', 'Health and Fitness', 'Messages', 'Photos or Videos', 'Audio Files', 'Files and docs', 'Calendar', 'Contacts', 'App Activity', 'Web Browsing', 'App Information and Performance', 'Device or Other Identifiers', 'None of the Above']
for i in range(start, end + 1, 1):
    key = df.columns[i]
    if key in datatypes:
        entry = 1
    else:
        entry = 2
    typeDict[key] = entry
answerDict = dict()
correct = dict()
order = dict()
for i in range(start, end+1, 1):
    for j in range(3, len(df.iloc[:, 1]), 1):
        name = df.iat[j, df.columns.get_loc("Q162")]
        question = df.columns[i]
        answer = df.iat[j, i]
        if "(Optional)" in descriptionsDict[question]: continue
        else:
            answerDict[(name, question)] = answer
            if answer == a_dict[descriptionsDict[question]]:
                correct[(name, question)] = 1
            else:
                correct[(name, question)] = 0
            if answer not in section1 and answer not in section2:
                continue
            if typeDict[question] == 1:
                index_given = section1.index(answer)
                index_expected = section1.index(a_dict[descriptionsDict[question]])
            else:
                index_given = section2.index(answer)
                index_expected = section2.index(a_dict[descriptionsDict[question]])
            if index_given < index_expected :
                order[(name, question)] = -1
            elif index_given < index_expected :
                order[(name, question)] = 0
            else :
                order[(name, question)] = 1
condition = dict()
for j in range(3, len(df.iloc[:, 1]), 1):
    condition[df.iat[j, df.columns.get_loc("Q162")]] = df.iat[j, -1]

In [41]:
main = []
print(df.columns.get_loc("Q101"))
for i in range(3, len(df.iloc[:, 0]), 1):
    currID = df.iat[i,df.columns.get_loc("Q162")]
    for j in range(start, df.columns.get_loc("Q101")+ 1):
        questionID = df.columns[j]
        if "(Optional)" in descriptionsDict[questionID]: continue
        else: main.append((questionID, currID))
main = sorted(main)
print(main)

121
[('Q100', '5e52af31a0002e2c6056b13f'), ('Q100', '5ea043fbbfbe190687467966'), ('Q100', '5eaf3e142627076426921b1b'), ('Q100', '5fe635082fcdc9404d6fc610'), ('Q100', '602d86345f6c24812bfc3a4e'), ('Q100', '603411dbcf15a5036c87fc2b'), ('Q100', '61123bdc9bdfe87e54a459a6'), ('Q100', '6124f281ce42cb9bf7ed738f'), ('Q100', '614dec67f6eb129c3a77defd'), ('Q101', '5e52af31a0002e2c6056b13f'), ('Q101', '5ea043fbbfbe190687467966'), ('Q101', '5eaf3e142627076426921b1b'), ('Q101', '5fe635082fcdc9404d6fc610'), ('Q101', '602d86345f6c24812bfc3a4e'), ('Q101', '603411dbcf15a5036c87fc2b'), ('Q101', '61123bdc9bdfe87e54a459a6'), ('Q101', '6124f281ce42cb9bf7ed738f'), ('Q101', '614dec67f6eb129c3a77defd'), ('Q108', '5e52af31a0002e2c6056b13f'), ('Q108', '5ea043fbbfbe190687467966'), ('Q108', '5eaf3e142627076426921b1b'), ('Q108', '5fe635082fcdc9404d6fc610'), ('Q108', '602d86345f6c24812bfc3a4e'), ('Q108', '603411dbcf15a5036c87fc2b'), ('Q108', '61123bdc9bdfe87e54a459a6'), ('Q108', '6124f281ce42cb9bf7ed738f'), ('Q108'

In [42]:
participantId = []
rowNames = []
for (questionID, currID) in main:
    participantId.append(currID)
    rowNames.append(questionID)
results = pd.DataFrame()
results.insert(0, 'Question ID', rowNames)
results.insert(1, 'Participant ID', participantId)
descriptions = list()
for i in range(len(rowNames)):
    descriptions.append(descriptionsDict[rowNames[i]])
types = list()
for i in range(len(rowNames)):
    types.append(typeDict[rowNames[i]])
answers = list()
for i in range(len(rowNames)):
    answers.append(answerDict[(participantId[i]), rowNames[i]])
correctness = list()
for i in range(len(rowNames)):
    correctness.append(correct[(participantId[i]), rowNames[i]])
conditionL = list()
for i in range(len(rowNames)):
    conditionL.append(condition[participantId[i]])
orderL = list()
for i in range(len(rowNames)):
    if order.get((participantId[i], rowNames[i]), 42) == 42:
        orderL.append("NaN")
    else:
        orderL.append(order[(participantId[i]), rowNames[i]])
results.insert(2, 'Description', descriptions)
results.insert(3, 'Answer', answers)
results.insert(4, 'Correctness', correctness)
results.insert(5, 'Question Type', types)
results.insert(6, 'Prompt Condition', conditionL)
results.insert(7, 'Order Difference', orderL)

print(results)


    Question ID            Participant ID               Description  \
0          Q100  5e52af31a0002e2c6056b13f  Firebase installation ID   
1          Q100  5ea043fbbfbe190687467966  Firebase installation ID   
2          Q100  5eaf3e142627076426921b1b  Firebase installation ID   
3          Q100  5fe635082fcdc9404d6fc610  Firebase installation ID   
4          Q100  602d86345f6c24812bfc3a4e  Firebase installation ID   
..          ...                       ...                       ...   
877         Q99  602d86345f6c24812bfc3a4e        Widevine Device ID   
878         Q99  603411dbcf15a5036c87fc2b        Widevine Device ID   
879         Q99  61123bdc9bdfe87e54a459a6        Widevine Device ID   
880         Q99  6124f281ce42cb9bf7ed738f        Widevine Device ID   
881         Q99  614dec67f6eb129c3a77defd        Widevine Device ID   

                              Answer  Correctness  Question Type  \
0                                NaN            0              2   
1          

In [43]:
results.to_excel("/Users/margarethe/Downloads/output.xlsx", sheet_name='Sheet_name_1')